# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [ ]:
# !pip uninstall torch -y

In [2]:
# 그 다음 다시 설치
!pip install -U "Pillow<12.0,>=9.0" "git+https://github.com/huggingface/transformers" "accelerate>=0.34.2" \
  "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pandas torch==2.8.0 torchvision

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7f5kjm4p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7f5kjm4p
  Resolved https://github.com/huggingface/transformers to commit 7a833d1ccd41673030c85107f65f454c0c3222f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torchvision-0.24.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.23.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 65.4 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/

In [4]:
# PyTorch (CUDA 12.1)
!pip install --upgrade --index-url https://download.pytorch.org/whl/cu121 \
  torch torchvision torchaudio

# 모델/로더 필수
!pip install --upgrade unsloth bitsandbytes transformers accelerate datasets pandas Pillow

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 75.0 MB/s  0:00:04:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 170.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 163.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 165.5 MB/s  0:00:0100:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 162.1 MB/s  0:00:0100:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 179.9 MB/s  0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-nccl-cu12━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [sympy]
    Found existing installation: nvidia-nccl-cu12 2.20.5━━━━━━ 1/6 [sympy]
    Uninstalling nvidia-nccl-cu12-2.20.5:━━━━━━━━━━━━

In [ ]:
# # 1. pip 자체를 최신 버전으로 업그레이드
# !pip install -U pip

# # 2. (핵심) Unsloth 라이브러리 설치
# # 노트북에서는 CUDA 12.1과 PyTorch 2.4.0 환경에 최적화된 버전을 설치합니다.
# !pip install -U "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"

# # 3. 나머지 의존성 패키지 설치
# !pip install -U "transformers>=4.57.0" "peft" "bitsandbytes" "accelerate" \
#   "datasets" "pandas" "Pillow<12.0,>=9.0"

In [ ]:
# (핵심) unsloth를 git main 브랜치에서 설치합니다.
# !pip install -U "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# # 나머지 패키지를 최신 버전으로 설치합니다.
# !pip install -U "transformers>=4.57.0" "peft" "bitsandbytes" "accelerate" \
#   "datasets" "pandas" "Pillow<12.0,>=9.0"

In [ ]:
# !pip -q install "Pillow<12.0,>=9.0" "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pandas torch torchvision --upgrade

# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [ ]:
# # 구글드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 압축 해제
# !unzip "/content/drive/My Drive/2025-ssafy-14" -d "/content/"

# 라이브러리, 데이터, 설정

In [1]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup,
    Qwen3VLForConditionalGeneration # Import the new model class
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "Qwen/Qwen3-VL-32B-Instruct" # Update MODEL_ID
IMAGE_SIZE = 384
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("/home/team067/train.csv")
test_df  = pd.read_csv("/home/team067/test.csv")


# 학습데이터 200개만 추출 -> 이아니라 전부할거임
# train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True) 

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


Device: cuda


# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [2]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor # Ensure Qwen3VLForConditionalGeneration is imported here

# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
base_model = Qwen3VLForConditionalGeneration.from_pretrained( # Use the new model class
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 14/14 [02:03<00:00,  8.79s/it]


trainable params: 67,108,864 || all params: 33,424,498,928 || trainable%: 0.2008


# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [3]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant.\n"
    "Return exactly one lowercase letter: a, b, c, or d.\n"
    "Do not output any other text, punctuation, parentheses, or spaces."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"Question: {question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n"
        "Choose the correct option and answer with one letter only: a b c d."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [4]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]},
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            # 안전장치: a/b/c/d 아닌 경우 기본값
            if gold not in ("a","b","c","d"):
                gold = "a"
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages}  # "image"는 없어도 OK


# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        input_texts, input_images = [], []
        noans_texts, noans_images = [], []
        has_gold = self.train

        for sample in batch:
            messages = sample["messages"]

            if has_gold:
                # 정답 포함(full) 텍스트
                full_text = self.processor.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=False
                )
                input_texts.append(full_text)
                # 정답 없는(no-answer) 텍스트: 마지막 assistant 메시지를 제거
                noans_msgs = messages[:-1]
                noans_text = self.processor.apply_chat_template(
                    noans_msgs, tokenize=False, add_generation_prompt=False
                )
                noans_texts.append(noans_text)

                # 이미지(둘 다 동일 객체 사용)
                # user 메시지 안의 이미지를 그대로 넘겨야 하므로, 첫 user의 image만 추출
                # 다중이미지면 필요한 만큼 추출 로직 확장
                for c in messages:
                    if c["role"] == "user":
                        for item in c["content"]:
                            if item["type"] == "image":
                                img = item["image"]
                                break
                        break
                input_images.append(img)
                noans_images.append(img)
            else:
                # 평가/추론: 생성 프롬프트 필요
                gen_text = self.processor.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
                input_texts.append(gen_text)
                for c in messages:
                    if c["role"] == "user":
                        for item in c["content"]:
                            if item["type"] == "image":
                                img = item["image"]
                                break
                        break
                input_images.append(img)

        # 토크나이즈
        enc_full = self.processor(
            text=input_texts,
            images=input_images,
            padding=True,
            return_tensors="pt",
        )

        if self.train:
            enc_noans = self.processor(
                text=noans_texts,
                images=noans_images,
                padding=True,
                return_tensors="pt",
            )
            input_ids = enc_full["input_ids"]
            labels = input_ids.clone()

            # 마스킹: no-answer 길이까지 -100
            noans_lens = (enc_noans["attention_mask"].sum(dim=1)).tolist()
            for i, L in enumerate(noans_lens):
                labels[i, :L] = -100

            enc_full["labels"] = labels

        return enc_full


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [5]:
# 검증용 데이터 분리
split = int(len(train_df)*0.9)
train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [7]:
from tqdm.auto import tqdm
import math, torch
from torch.nn.utils import clip_grad_norm_

EPOCHS = 1
GRAD_ACCUM = 4
LR = 5e-5

# 선택: TF32 가속 (Ampere+)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True

model = model.to(model.device)  # device 변수 대신 model.device 사용 권장
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

num_update_steps_per_epoch = math.ceil(len(train_loader) / GRAD_ACCUM)
num_training_steps = EPOCHS * num_update_steps_per_epoch
num_warmup_steps = max(1, int(num_training_steps * 0.03))

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)

amp_dtype = torch.bfloat16  # GPU가 bf16 미지원이면 torch.float16
scaler = torch.cuda.amp.GradScaler(enabled=True)

global_step = 0
for epoch in range(EPOCHS):
    model.train()
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")

    optimizer.zero_grad(set_to_none=True)  # ← 축적 시작 전에 한 번

    for step, batch in enumerate(progress_bar, start=1):
        # model.device 기준 이동 (텐서만)
        batch = {k: (v.to(model.device) if hasattr(v, "to") else v) for k, v in batch.items()}

        with torch.cuda.amp.autocast(dtype=amp_dtype):
            outputs = model(**batch)           # batch에는 labels가 포함되어 있어야 함(DataCollator에서 처리)
            loss = outputs.loss / GRAD_ACCUM

        scaler.scale(loss).backward()
        running += loss.item()

        if step % GRAD_ACCUM == 0:
            # 클리핑(권장 0.5~1.0)
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), max_norm=1.0)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0

    # ----- Validation -----
    model.eval()
    val_loss, val_steps = 0.0, 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=amp_dtype):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k: (v.to(model.device) if hasattr(v, "to") else v) for k, v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")

    model.train()  # 다음 에폭 대비



/tmp/ipykernel_5134/3912965950.py:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)
Epoch 1 [train]:   0%|          | 0/3498 [00:00<?, ?batch/s]

/tmp/ipykernel_5134/3912965950.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=amp_dtype):
Epoch 1 [train]: 100%|██████████| 3498/3498 [1:24:26<00:00,  1.45s/batch, loss=0.004]
/tmp/ipykernel_5134/3912965950.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=amp_dtype):
Epoch 1 [valid]: 100%|██████████| 389/389 [03:17<00:00,  1.97batch/s]

[Epoch 1] valid loss 0.0393


In [8]:

# 모델 저장
SAVE_DIR = "qwen3_vl_32b_lora_commentLee"
# LoRA 사용 중이면: 어댑터만 저장
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved:", SAVE_DIR)


Saved: qwen3_vl_32b_lora_commentLee


# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [10]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
import re

def extract_choice(text: str) -> str:
    if not text:
        return "a"
    text = text.strip().lower()

    # 가장 먼저 정확히 a/b/c/d 단독 토큰 매칭
    m = re.search(r'\b([abcd])\b', text)
    if m:
        return m.group(1)

    # 괄호/콜론/닫힘기호가 붙은 형태까지 허용: (a), a), a:, - a
    m = re.search(r'[\(\[\{<\s\-:]*([abcd])[\)\]\}>:.,\s]*$', text)
    if m:
        return m.group(1)

    # 마지막 라인/토큰에서 다시 한 번 안전 검색
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if lines:
        last = lines[-1]
        m = re.search(r'\b([abcd])\b', last)
        if m:
            return m.group(1)
        for tok in last.split():
            if tok in ("a","b","c","d"):
                return tok

    return "a"  # 최후의 안전장치

# 추론을 위해 모든 레이어 활성화
model.eval()
preds = []

# (선택) pad_token 설정: 경고 방지용 — 기존 동작엔 영향 없음
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id

for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":user_text}
        ]},
    ]

    # 템플릿 적용(토크나이즈는 generate에서)
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
      out_ids = model.generate(
          **inputs,
          max_new_tokens=2,          # 원래 값 유지(원하시면 1로 줄여도 됨)
          do_sample=False,
          eos_token_id=processor.tokenizer.eos_token_id,
      )

    # 🔧 입력 길이 이후(신규 토큰)만 슬라이스
    gen_part = out_ids[:, inputs["input_ids"].shape[1]:]
    output_text = processor.batch_decode(
        gen_part, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]

    choice = extract_choice(output_text)
    # 처음 테스트할때는 주석해제해서 응답제대로나오는지 확인
    # print(f"[{i}] output_text:", repr(output_text))
    # print(f"[{i}] extract_choice:", choice)
    preds.append(choice)


    

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")

# ===== abcd 선택 횟수 출력 =====
print("\n--- 선택지별 응답 횟수 ---")
# .sort_index()를 사용하면 a, b, c, d 순서로 정렬되어 나옵니다.
print(submission["answer"].value_counts().sort_index())
# ========================================================


Inference:   0%|          | 0/3887 [00:00<?, ?sample/s]

Inference: 100%|██████████| 3887/3887 [50:23<00:00,  1.29sample/s]

Saved submission.csv

--- 선택지별 응답 횟수 ---
answer
a     958
b     956
c    1006
d     967
Name: count, dtype: int64


In [11]:
# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("submission2.csv", index=False)
print("Saved submission.csv")

Saved submission.csv


In [ ]:
# 모델 응답 예시
print(output_text)